In [1]:
import pandas as pd
from datetime import datetime
from datetime import timedelta
transaction_data = pd.read_csv('C:/Users/SJ/Desktop/MSBA/Marketing/transaction_data.csv')
cust_data = pd.read_csv('C:/Users/SJ/Desktop/MSBA/Marketing/cust_data.csv')
transaction_data['Order Date'] = pd.to_datetime(transaction_data['Order Date'])

In [2]:
# Define the treatment start date
treatment_start_date = datetime.strptime("2019-03-02", "%Y-%m-%d")

# Define the pre-treatment and post-treatment date ranges
pre_treatment_end_date = treatment_start_date - timedelta(days=1)
pre_treatment_start_date = pre_treatment_end_date - timedelta(days=59)
post_treatment_start_date = treatment_start_date
post_treatment_end_date = treatment_start_date + timedelta(days=59)

# Filter the transactions for the pre-treatment and post-treatment periods
pre_treatment_transactions = transaction_data[
    (transaction_data['Order Date'] >= pre_treatment_start_date) &
    (transaction_data['Order Date'] <= pre_treatment_end_date)
]
post_treatment_transactions = transaction_data[
    (transaction_data['Order Date'] >= post_treatment_start_date) &
    (transaction_data['Order Date'] <= post_treatment_end_date)
]

# Calculate the total profit per customer for each period
pre_treatment_profit = pre_treatment_transactions.groupby('Customer ID')['Profit'].sum().reset_index()
post_treatment_profit = post_treatment_transactions.groupby('Customer ID')['Profit'].sum().reset_index()

# Rename the columns to reflect pre-treatment and post-treatment profits
pre_treatment_profit.rename(columns={'Profit': 'Profit 60 Days Before Treatment'}, inplace=True)
post_treatment_profit.rename(columns={'Profit': 'Profit 60 Days After Treatment'}, inplace=True)

# Merge the customer data with the pre-treatment and post-treatment profits
combined_data = cust_data.merge(pre_treatment_profit, on='Customer ID', how='left')
combined_data = combined_data.merge(post_treatment_profit, on='Customer ID', how='left')

# Replace NaN values with zeros if there were no profits in the period
combined_data.fillna(0, inplace=True)

In [3]:
combined_data

,Customer ID,State,Treatment Group,Profit 60 Days Before Treatment,Profit 60 Days After Treatment
0,0,VA,No Mail,0.00,0.00
1,1,VA,Mail,27.94,0.00
2,2,VA,Mail,530.79,126.26
3,3,OH,Mail,0.00,0.00
4,4,OH,Mail,54.72,99.91
...,...,...,...,...,...
11995,11995,GA,No Mail,0.00,115.05
11996,11996,GA,No Mail,146.98,68.42
11997,11997,OH,Mail,0.00,0.00
11998,11998,OH,Mail,6.92,35.21


In [4]:
# Mean & SD for Profit 60 Days Before Treatment
mean_before_treatment = combined_data['Profit 60 Days Before Treatment'].mean()
std_before_treatment = combined_data['Profit 60 Days Before Treatment'].std()

# Mean & SD for Profit 60 Days After Treatment
mean_after_treatment = combined_data['Profit 60 Days After Treatment'].mean()
std_after_treatment = combined_data['Profit 60 Days After Treatment'].std()

print(f"Mean (Profit 60 Days Before Treatment): {mean_before_treatment}")
print(f"Standard Deviation (Profit 60 Days Before Treatment): {std_before_treatment}")
print(f"Mean (Profit 60 Days After Treatment): {mean_after_treatment}")
print(f"Standard Deviation (Profit 60 Days After Treatment): {std_after_treatment}")

Mean (Profit 60 Days Before Treatment): 77.33159666666667
Standard Deviation (Profit 60 Days Before Treatment): 229.00680235957972
Mean (Profit 60 Days After Treatment): 119.34988583333335
Standard Deviation (Profit 60 Days After Treatment): 337.1188200056834


In [5]:
# Constants
cost_per_catalog = 20
total_active_customer_base = 312000
number_of_customers_in_treatment_group = 12000

# Filter Mail Group
mail_group = combined_data[combined_data['Treatment Group'] == 'Mail']

# Calculate the total profit before and after the treatment for the treatment group
total_profit_before = mail_group['Profit 60 Days Before Treatment'].sum()
total_profit_after = mail_group['Profit 60 Days After Treatment'].sum()

# The treatment effect is the difference in the totals
total_treatment_effect = total_profit_after - total_profit_before
per_customer_treatment_effect = total_treatment_effect / number_of_customers_in_treatment_group

# Calculate the net profit per customer in the treatment group (corrected)
net_profit_per_customer_treatment = per_customer_treatment_effect - cost_per_catalog

# Calculate the number of customers in the remaining population (corrected)
remaining_population = total_active_customer_base - number_of_customers_in_treatment_group

# Calculate the expected gain over the remaining population (corrected)
expected_gain_over_remaining_population = net_profit_per_customer_treatment * remaining_population

total_treatment_effect, per_customer_treatment_effect, net_profit_per_customer_treatment, expected_gain_over_remaining_population

(319037.47000000003, 26.586455833333336, 6.586455833333336, 1975936.7500000007)

In [6]:
from scipy.stats import chi2_contingency

# Create a contingency table of the counts of customers in each State by Treatment Group
contingency_table = combined_data.pivot_table(index='State', columns='Treatment Group', aggfunc='size', fill_value=0)

# Perform the Chi-Square Test of Independence
chi2_stat, p_value, dof, expected = chi2_contingency(contingency_table)

# Report the p-value
p_value

0.8271959154190364

In [7]:
from scipy.stats import ttest_ind

# Filter the data for the Mail and No Mail groups
mail_group = combined_data[combined_data['Treatment Group'] == 'Mail']
no_mail_group = combined_data[combined_data['Treatment Group'] == 'No Mail']

# Perform the two-sided t-test on the 'Profit 60 Days Before Treatment'
t_stat, p_value = ttest_ind(mail_group['Profit 60 Days Before Treatment'], no_mail_group['Profit 60 Days Before Treatment'], equal_var=False)

# Report the p-value
p_value

0.27482743019870987

In [8]:
# Calculate the mean profit 60 days after treatment for both groups
mean_profit_mail = combined_data[combined_data['Treatment Group'] == 'Mail']['Profit 60 Days After Treatment'].mean()
mean_profit_no_mail = combined_data[combined_data['Treatment Group'] == 'No Mail']['Profit 60 Days After Treatment'].mean()

# Estimate the Average Treatment Effect (ATE)
ate = mean_profit_mail - mean_profit_no_mail

# b. Determine Statistical Significance
# Perform a two-sample t-test
t_stat, p_value = ttest_ind(
    combined_data[combined_data['Treatment Group'] == 'Mail']['Profit 60 Days After Treatment'],
    combined_data[combined_data['Treatment Group'] == 'No Mail']['Profit 60 Days After Treatment'],
    equal_var=False  # Assuming variances are not equal
)

# Calculate the standard error of the mean difference
n_mail = combined_data[combined_data['Treatment Group'] == 'Mail']['Profit 60 Days After Treatment'].count()
n_no_mail = combined_data[combined_data['Treatment Group'] == 'No Mail']['Profit 60 Days After Treatment'].count()
std_mail = combined_data[combined_data['Treatment Group'] == 'Mail']['Profit 60 Days After Treatment'].std()
std_no_mail = combined_data[combined_data['Treatment Group'] == 'No Mail']['Profit 60 Days After Treatment'].std()
standard_error = ((std_mail**2) / n_mail + (std_no_mail**2) / n_no_mail) ** 0.5

ate, p_value, standard_error

(17.743285000000014, 0.003938274308826571, 6.153044015728518)

In [9]:
# Constants
cost_per_catalog = 20  # Cost to produce and mail each catalog
total_active_customer_base = 312000  # Total number of customers in the active customer base
number_of_customers_in_experiment = 12000  # Number of customers in the treatment group

ate = 17.74  # Average Treatment Effect from previous analysis

# Calculate the Net Gain Per Customer by subtracting the cost of the catalog from the ATE
net_gain_per_customer = ate - cost_per_catalog

# Calculate the number of customers in the remaining population
remaining_population = total_active_customer_base - number_of_customers_in_experiment

# Calculate the Total Expected Gain for the remaining population
total_expected_gain = net_gain_per_customer * remaining_population

# Print the results
print(f"Total Expected Gain: {total_expected_gain}")

Total Expected Gain: -678000.0000000005


In [10]:
import pandas as pd
from scipy.stats import ttest_ind

# a. Estimate the Treatment Effect using the DiD Method
avg_profit_pre_mail = combined_data[combined_data['Treatment Group'] == 'Mail']['Profit 60 Days Before Treatment'].mean()
avg_profit_post_mail = combined_data[combined_data['Treatment Group'] == 'Mail']['Profit 60 Days After Treatment'].mean()
avg_profit_pre_no_mail = combined_data[combined_data['Treatment Group'] == 'No Mail']['Profit 60 Days Before Treatment'].mean()
avg_profit_post_no_mail = combined_data[combined_data['Treatment Group'] == 'No Mail']['Profit 60 Days After Treatment'].mean()

diff_mail = avg_profit_post_mail - avg_profit_pre_mail
diff_no_mail = avg_profit_post_no_mail - avg_profit_pre_no_mail
did_estimator = diff_mail - diff_no_mail

# b. Determine Statistical Significance
# Calculating standard errors and performing a t-test
mail_diff = combined_data[combined_data['Treatment Group'] == 'Mail']['Profit 60 Days After Treatment'] - combined_data[combined_data['Treatment Group'] == 'Mail']['Profit 60 Days Before Treatment']
no_mail_diff = combined_data[combined_data['Treatment Group'] == 'No Mail']['Profit 60 Days After Treatment'] - combined_data[combined_data['Treatment Group'] == 'No Mail']['Profit 60 Days Before Treatment']
t_stat_did, p_value_did = ttest_ind(mail_diff, no_mail_diff, equal_var=False)
se_did = ((mail_diff.std() ** 2 / len(mail_diff)) + (no_mail_diff.std() ** 2 / len(no_mail_diff))) ** 0.5

# c. Decision on Launching a Full-Scale Marketing Campaign
# This decision is based on the significance (p-value) and magnitude of the DiD estimator.
# If p_value_did < 0.05 and did_estimator is positive and sufficiently large, launching a campaign could be considered.

# d. Expected Gain on the Remaining Population
cost_per_catalog = 20
total_active_customer_base = 312000
number_of_customers_in_experiment = 12000

net_gain_per_customer_did = did_estimator - cost_per_catalog
remaining_population_did = total_active_customer_base - number_of_customers_in_experiment
total_expected_gain_did = net_gain_per_customer_did * remaining_population_did

# Printing results for review
print(f"DiD Estimator: {did_estimator}")
print(f"P-value: {p_value_did}")
print(f"Standard Error: {se_did}")
print(f"Net Gain Per Customer: {net_gain_per_customer_did}")
print(f"Total Expected Gain: {total_expected_gain_did}")

DiD Estimator: 22.309245000000004
P-value: 1.362663769418418e-07
Standard Error: 4.230377997113267
Net Gain Per Customer: 2.309245000000004
Total Expected Gain: 692773.5000000013


In [11]:
# Import necessary libraries
import numpy as np
from scipy.stats import ttest_ind

# Constants
cost_per_catalog = 20
total_active_customer_base = 312000 - 12000  # Adjusting for the number of customers in the experiment
number_of_customers_in_experiment = 12000

# Compute ATE for each state separately and evaluate the possibility of launching a marketing campaign
ates = {}
expected_gains = {}
for state in ['GA', 'OH', 'VA']:  # Assuming these are the three states in the dataset
    # Subset data for the state
    state_data = combined_data[combined_data['State'] == state]
    
    # Calculate mean profits for Mail and No Mail groups before and after the treatment
    pre_mail = state_data[state_data['Treatment Group'] == 'Mail']['Profit 60 Days Before Treatment'].mean()
    post_mail = state_data[state_data['Treatment Group'] == 'Mail']['Profit 60 Days After Treatment'].mean()
    pre_no_mail = state_data[state_data['Treatment Group'] == 'No Mail']['Profit 60 Days Before Treatment'].mean()
    post_no_mail = state_data[state_data['Treatment Group'] == 'No Mail']['Profit 60 Days After Treatment'].mean()
    
    # Calculate ATE for the state
    ate = (post_mail - pre_mail) - (post_no_mail - pre_no_mail)
    ates[state] = ate
    
    # Estimate the number of customers in the state within the remaining population
    proportion_in_sample = len(state_data) / number_of_customers_in_experiment
    estimated_customers_in_state = proportion_in_sample * total_active_customer_base
    
    # Calculate expected gain for the state
    expected_gain = (ate - cost_per_catalog) * estimated_customers_in_state if ate > cost_per_catalog else 0
    expected_gains[state] = expected_gain

ates, expected_gains

({'GA': 37.260262479112896, 'OH': 23.40006304285548, 'VA': 2.8997820729018073},
 {'GA': 1866265.8805540819, 'OH': 348676.46504482935, 'VA': 0})